In [1]:
import pandas as pd
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize

In [2]:
# Import data
df = pd.read_csv('preprocessed_amazon.csv')
df.head(2)

,Category,Product Title,Product Description,Price,Conditions,Description Length
0,Skin Care | Lee Posh,Lee posh Lactic Acid 60% Anti ageing Pigmenta...,PROFESSIONAL GRADE Face Peel: this peel stimul...,11.19,Very bad,47
1,Skin Care | Generic,Generic 1 Pc brand snail eye cream remove dar...,"Use: eye, item type: cream, net wt: 20g, gzzz:...",14.59,Very good,64


In [3]:
# Import data
df = pd.read_csv('preprocessed_amazon.csv')

# Tokenization and Vectorization using TF-IDF
tfidf_vectorizer = TfidfVectorizer()
X_tfidf = tfidf_vectorizer.fit_transform(df['Product Description'])

# Tokenization and Vectorization using Word2Vec
tokenized_text = [word_tokenize(text) for text in df['Product Description']]
word2vec_model = Word2Vec(sentences=tokenized_text, vector_size=100, window=5, min_count=1, sg=0)
X_word2vec = [word2vec_model.wv[word] for word in tokenized_text]


In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors
import pandas as pd

# Vectorize the product descriptions
tfidf_vectorizer = TfidfVectorizer()
X = tfidf_vectorizer.fit_transform(df['Product Description'])

# Define the KNN model
knn_model = NearestNeighbors(n_neighbors=5, metric='cosine')
knn_model.fit(X)

# Define a function to find similar posts
def find_similar_posts(product_description, k=15):
    # Vectorize the input product description
    description_vector = tfidf_vectorizer.transform([product_description])
    # Find the indices of the k nearest neighbors
    _, indices = knn_model.kneighbors(description_vector)
    # Return the titles of the most similar posts
    return df.iloc[indices[0]]['Product Title']

# Example usage
input_description = "white FLOWER"
similar_posts = find_similar_posts(input_description)
print(similar_posts)


6317      Asquith & Somerset Gardenia Flower Moisturizi...
16771     Mystique Hills Organic Neem Flower Herbal Tea...
7664      Ancient Flower - Natural Vitamin C and Lavend...
16403     Ancient Flower - Natural Vitamin C and Lavend...
468       Ancient Flower - Natural Vitamin C and Lavend...
Name: Product Title, dtype: object


In [5]:
from sklearn.neighbors import NearestNeighbors
import numpy as np

# Fit the KNN model using TF-IDF vectors
knn_tfidf = NearestNeighbors(n_neighbors=15, metric='cosine')
knn_tfidf.fit(X_tfidf)

# Fit the KNN model using Word2Vec vectors
X_word2vec_array = np.array([np.mean(vec, axis=0) for vec in X_word2vec])
knn_word2vec = NearestNeighbors(n_neighbors=15, metric='cosine')
knn_word2vec.fit(X_word2vec_array)

def find_similar_posts(input_text, vectorizer, knn_model, top_k=15):
    if vectorizer == 'tfidf':
        input_vector = tfidf_vectorizer.transform([input_text])
    elif vectorizer == 'word2vec':
        words = word_tokenize(input_text)
        word_vectors = [word2vec_model.wv[word] for word in words if word in word2vec_model.wv]
        if len(word_vectors) == 0:
            return []
        input_vector = np.mean(word_vectors, axis=0).reshape(1, -1)
    
    if vectorizer == 'tfidf':
        _, indices = knn_tfidf.kneighbors(input_vector)
    elif vectorizer == 'word2vec':
        _, indices = knn_word2vec.kneighbors(input_vector)
    
    return df.iloc[indices[0]]['Product Title']

# Example usage
input_text = "purple phonecase"
similar_posts_tfidf = find_similar_posts(input_text, 'tfidf', knn_tfidf)
similar_posts_word2vec = find_similar_posts(input_text, 'word2vec', knn_word2vec)

print("Similar posts using TF-IDF:")
print(similar_posts_tfidf)
print("\nSimilar posts using Word2Vec:")
print(similar_posts_word2vec)


Similar posts using TF-IDF:
13952     Tootsie Rolls Frooties Grape Candy (360 Count...
7987                Tec Italy - Lumina Purple Conditioner 
2548      Storeway Nike Basic Purple EDT Deodorant Body...
5093      60ml , purple : MSmask Travel Bottle Squeezin...
15091     Purple : ecoLove Purple Collection Conditione...
11987     stephblack Simple Bathing Strap with Exfoliat...
10601     Avon Little Gold Dress Combo (EDP + Skin Soft...
364       Reiko Flexible Long Arms Hand Free Phone Holder 
14215     ddp acrylic nail pincher tool multi function ...
2369      Fashlady™ Green: Elecool 1 Pair Spa Gel Socks...
4471      Lili DIY Nails Rhinestone 3D Acrylic Crystal ...
6781      Ubervia Pack of 1 100g Multicolor Bath Ball H...
6407      Bombshell By Victorias Secret For Women Body ...
10195     Generic Purple, L : Yel 2 pcs Top Basketball ...
7825      Generic Pink: pro Makeup Candy Color Moisturi...
Name: Product Title, dtype: object

Similar posts using Word2Vec:
2643      Genuine Kar

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors
import numpy as np
from nltk.tokenize import word_tokenize
import gensim.downloader as api

# # Fit the TF-IDF vectorizer
# tfidf_vectorizer = TfidfVectorizer()
# X_tfidf = tfidf_vectorizer.fit_transform(df)

# Vectorize the product descriptions
tfidf_vectorizer = TfidfVectorizer()
X = tfidf_vectorizer.fit_transform(df['Product Description'])

# Fit the KNN model using TF-IDF vectors
knn_tfidf = NearestNeighbors(n_neighbors=15, metric='cosine')
knn_tfidf.fit(X_tfidf)

# Fit the KNN model using GloVe vectors
def get_glove_vector(word):
    try:
        return glove_model[word]
    except KeyError:
        return None

def get_mean_glove_vectors(words):
    vectors = [get_glove_vector(word) for word in words]
    vectors = [vec for vec in vectors if vec is not None]
    if len(vectors) == 0:
        return None
    return np.mean(vectors, axis=0).reshape(1, -1)

def get_glove_vectorized_data(X):
    vectors = [get_mean_glove_vectors(word_tokenize(str(text))) for text in X['Product Title']]
    valid_vectors = [vec for vec in vectors if vec is not None]
    return np.concatenate(valid_vectors) if valid_vectors else np.array([])

X_glove_array = get_glove_vectorized_data(df)
knn_glove = NearestNeighbors(n_neighbors=15, metric='cosine')
knn_glove.fit(X_glove_array)



X_glove_array = get_glove_vectorized_data(df)
knn_glove = NearestNeighbors(n_neighbors=15, metric='cosine')
knn_glove.fit(X_glove_array)

def find_similar_posts(input_text, vectorizer, knn_model, top_k=15):
    if vectorizer == 'tfidf':
        input_vector = tfidf_vectorizer.transform([input_text])
    elif vectorizer == 'glove':
        input_vector = get_mean_glove_vectors(word_tokenize(input_text))
        if input_vector is None:
            return []
    
    if vectorizer == 'tfidf':
        _, indices = knn_tfidf.kneighbors(input_vector)
    elif vectorizer == 'glove':
        _, indices = knn_glove.kneighbors(input_vector)
    
    return df.iloc[indices[0]]['Product Title']

# Example usage
input_text = "purple phonecase"
similar_posts_tfidf = find_similar_posts(input_text, 'tfidf', knn_tfidf)
similar_posts_glove = find_similar_posts(input_text, 'glove', knn_glove)

print("Similar posts using TF-IDF:")
print(similar_posts_tfidf)
print("\nSimilar posts using GloVe:")
print(similar_posts_glove)

NameError: name 'glove_model' is not defined

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors
import numpy as np
from nltk.tokenize import word_tokenize
import gensim.downloader as api

# Load the GloVe model
glove_model = api.load('glove-wiki-gigaword-100')

# Vectorize the product descriptions
tfidf_vectorizer = TfidfVectorizer()
X_tfidf = tfidf_vectorizer.fit_transform(df['Product Description'])

# Fit the KNN model using TF-IDF vectors
knn_tfidf = NearestNeighbors(n_neighbors=15, metric='cosine')
knn_tfidf.fit(X_tfidf)

# Fit the KNN model using Word2Vec vectors
X_word2vec_array = np.array([np.mean(vec, axis=0) for vec in X_word2vec])
knn_word2vec = NearestNeighbors(n_neighbors=15, metric='cosine')
knn_word2vec.fit(X_word2vec_array)

# Fit the KNN model using GloVe vectors
def get_glove_vector(word):
    try:
        return glove_model[word]
    except KeyError:
        return None

def get_mean_glove_vectors(words):
    vectors = [get_glove_vector(word) for word in words]
    vectors = [vec for vec in vectors if vec is not None]
    if len(vectors) == 0:
        return None
    return np.mean(vectors, axis=0).reshape(1, -1)

def get_glove_vectorized_data(X):
    vectors = [get_mean_glove_vectors(word_tokenize(str(text))) for text in X['Product Title']]
    valid_vectors = [vec for vec in vectors if vec is not None]
    return np.concatenate(valid_vectors) if valid_vectors else np.array([])

X_glove_array = get_glove_vectorized_data(df)
knn_glove = NearestNeighbors(n_neighbors=15, metric='cosine')
knn_glove.fit(X_glove_array)

def find_similar_posts(input_text, vectorizer, knn_model, top_k=15):
    if vectorizer == 'tfidf':
        input_vector = tfidf_vectorizer.transform([input_text])
    elif vectorizer == 'word2vec':
        words = word_tokenize(input_text)
        word_vectors = [word2vec_model.wv[word] for word in words if word in word2vec_model.wv]
        if len(word_vectors) == 0:
            return []
        input_vector = np.mean(word_vectors, axis=0).reshape(1, -1)
    elif vectorizer == 'glove':
        input_vector = get_mean_glove_vectors(word_tokenize(input_text))
        if input_vector is None:
            return []

    if vectorizer == 'tfidf':
        _, indices = knn_tfidf.kneighbors(input_vector)
    elif vectorizer == 'word2vec':
        _, indices = knn_word2vec.kneighbors(input_vector)
    elif vectorizer == 'glove':
        _, indices = knn_glove.kneighbors(input_vector)
    
    similar_indices = indices.flatten()[:top_k]
    similar_posts = [df.iloc[idx]['Product Title'] for idx in similar_indices]
    
    return similar_posts

# Example usage
input_text = "purple phonecase"
similar_posts_tfidf = find_similar_posts(input_text, 'tfidf', knn_tfidf)
similar_posts_word2vec = find_similar_posts(input_text, 'word2vec', knn_word2vec)
similar_posts_glove = find_similar_posts(input_text, 'glove', knn_glove)

print("Similar posts using TF-IDF:")
for post in similar_posts_tfidf:
    print(post)

print("\nSimilar posts using Word2Vec:")
for post in similar_posts_word2vec:
    print(post)

print("\nSimilar posts using GloVe:")
for post in similar_posts_glove:
    print(post)


Similar posts using TF-IDF:
 Tootsie Rolls Frooties Grape Candy (360 Count), 38.8Oz 
 Tec Italy - Lumina Purple Conditioner 
 Storeway Nike Basic Purple EDT Deodorant Body Spray for Women 200ml 
 60ml , purple : MSmask Travel Bottle Squeezing Bath Lotion Cosmetic Skincare Refillable None Leaking 
 Purple : ecoLove Purple Collection Conditioner for colored and very dry hair - Blueberry, Grape & Lavender. 100% VEGAN (17.6 Oz) 
 stephblack Simple Bathing Strap with Exfoliating Bathing Ball Back Sponge Nylon Brush Scrubbing Back Wipe(None Yellow) 
 Avon Little Gold Dress Combo (EDP + Skin Softener) 
 Reiko Flexible Long Arms Hand Free Phone Holder 
 ddp acrylic nail pincher tool multi function cuticle pusher tweezer magic wand purple 
 Fashlady™ Green: Elecool 1 Pair Spa Gel Socks Moisturize Soften Repair Cracked Skin Gel Socks Skin Moisturizing Gel Spa Socks Manicure Tools 
 Lili DIY Nails Rhinestone 3D Acrylic Crystal Glitter jewelry Nail Art Decorations Makeup Tools: 036 
 Ubervia Pack 